In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np

import xgboost as xgb
import pickle

In [2]:
r16_tab1 = pd.read_json('C:\\Users\\user\\deep_learning\\MDMS\\DataCompters\\R16\\R16_alm_2022-11-15_19\\R16_alm_2022-11-15.log', lines=True)

MemoryError: 

In [ ]:
r16_tab1=r16_tab1.loc[r16_tab1.type_mesure.isin(['courant', 'courant_hta', 'desequilibre_courant', 
                                                 'desequilibre_tension', 'tension_n', 'tension_ph',
                                                 'tension_ph_hta'
                                                 ])]
                                               

In [ ]:
mon_dictionnaire = {'courant': int(1), 'courant_hta': int(2), 'desequilibre_courant':int(3),  
                    'tension_n':int(4), 'tension_ph':int(5), 'tension_ph_hta':int(6), 
                    'desequilibre_tension':int(7)}


r16_tab1 = r16_tab1.set_index('date_heure_mesure')
r16_tab1.index = pd.to_datetime(r16_tab1.index)


def create_features(r16_tab1):
            """
            Create time series features based on time series index.
            """
            r16_tab1 = r16_tab1.copy()
            r16_tab1['heure_mesure'] = r16_tab1.index.hour
            r16_tab1['minute_mesure'] = (r16_tab1.index.minute)
            r16_tab1['seconde_mesure'] = r16_tab1.index.second
            # r16_tab1['depart'] =r16_tab1.depart
            r16_tab1['type_mesure'] = r16_tab1.type_mesure
            r16_tab1['phase'] = r16_tab1.phase
            return r16_tab1

for type_mesure, encoder in mon_dictionnaire.items():
        # print("l'élément de clé", type_mesure, "vaut", encoder)
        r16_tab1_2 = r16_tab1.copy()

        r16_tab1_2['type_mesure'] = r16_tab1_2['type_mesure'].map({
                             type_mesure : encoder,
                            
                             },
                             na_action=None)

        r16_tab1_2['phase']= r16_tab1_2['phase'].replace(['N'],[4])
        # set(r16_tab1.phase)
        r16_tab1_2 = r16_tab1_2.dropna(subset='type_mesure')
        r16_tab1_2['phase']=r16_tab1_2['phase'].astype(int)


        train = r16_tab1_2
        test = r16_tab1_2

        r16_tab1_2 = create_features(r16_tab1_2)
        train = create_features(train)
        test = create_features(test)

        FEATURES = ['type_mesure', 'phase', 'heure_mesure', 'minute_mesure', 'seconde_mesure']
        TARGET = 'mesure'

        X_train = train[FEATURES]
        y_train = train[TARGET]

        X_test = test[FEATURES]
        y_test = test[TARGET]

        reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                            n_estimators=10000,
                            early_stopping_rounds=50,
                            objective='reg:linear',
                            max_depth=3,
                            learning_rate=0.001)
        reg.fit(X_train, y_train,
                eval_set=[(X_train, y_train), (X_test, y_test)],
                verbose=100)
                
        filename = f'r16_Tfo1_{type_mesure}'

        with open(f'{filename}.pkl', "wb") as f:
            pickle.dump(reg, f)

In [ ]:
mon_dictionnaire = {'courant': int(1), 'courant_hta': int(2), 'desequilibre_courant':int(3),  
                    'tension_n':int(4), 'tension_ph':int(5), 'tension_ph_hta':int(6), 
                    'desequilibre_tension':int(7)}


In [ ]:
r16_tab1

In [ ]:
test = r16_tab1.copy()

test['type_mesure'] = test['type_mesure'].map({
                             'courant' : int(1),                           
                             },
                             na_action=None)

test['phase']= test['phase'].replace(['N'],[4])
        # set(r16_tab1.phase)
test = test.dropna(subset='type_mesure')
test['phase']=test['phase'].astype(int)
test = create_features(test)
X_test = test[FEATURES]

with open('r16_Tfo1_courant.pkl', "rb") as f:
    reg = pickle.load(f)

test['prediction'] = reg.predict(X_test)
test.iloc[0:60]

In [ ]:
voir_test = test.loc[(test['type_mesure'] == 1) & (test['phase'] == 1)]

a = voir_test.index
b = voir_test.prediction
# c = voir_test.index
# d = voir_test.prediction

# plt.figure(figsize=(12, 8))
# plt.subplot(3,1,1)
plt.plot(a, b, c='yellow')
# plt.plot(c,d, c='yellow')
plt.show

In [ ]:
# r16_tab1 = r16_tab1.set_index('date_heure_mesure')
# r16_tab1.index = pd.to_datetime(r16_tab1.index)

def create_features(r16_tab1):
            """
            Create time series features based on time series index.
            """
            r16_tab1 = r16_tab1.copy()
            r16_tab1['heure_mesure'] = r16_tab1.index.hour
            r16_tab1['minute_mesure'] = (r16_tab1.index.minute)
        #     r16_tab1['seconde_mesure'] = r16_tab1.index.second
            r16_tab1['depart'] =r16_tab1.depart
            r16_tab1['type_mesure'] = r16_tab1.type_mesure
            r16_tab1['phase'] = r16_tab1.phase
            return r16_tab1

test = create_features(r16_tab1)

FEATURES = ['depart', 'type_mesure', 'phase', 'heure_mesure', 'minute_mesure']
TARGET = 'mesure'

# X_train = train[FEATURES]
# y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

with open('r16_tab1_tdh_tension.pkl', "rb") as f:
    reg = pickle.load(f)

test['prediction'] = reg.predict(X_test)
test

In [ ]:
mere = test.copy()
mere['laisse'] = 'seconde_mesure'
test

In [ ]:
mon_dictionnaire = {'courant': int(1), 'courant_hta': int(2), 'desequilibre_courant':int(3),  
                    'tension_n':int(4), 'tension_ph':int(5), 'tension_ph_hta':int(6), 
                    'desequilibre_tension':int(7)}


In [ ]:
r16_tab1.loc[r16_tab1['type_mesure']== 'tdh_courant'].drop_duplicates(subset='depart')

In [ ]:
r16_tab1.loc[(r16_tab1['type_mesure'] == 'tdh_courant')]

In [ ]:
bt= r16_tab1.copy
bt
bt['type_mesure'].loc[bt['type_mesure']== 1] = eng.predict(bt)


In [ ]:
import itertools
import numpy as np
import pandas as pd
import random

a = [[1,2,3,4,5,6,7],[1,2,3,4]]
famille = list(itertools.product(*a))

my_data = []
for i in famille:
    my_data.append(list(i))

my_data = np.array(my_data)

column_names = ["type_mesure", "phase"]
data_df = pd.DataFrame(my_data, columns=column_names)
data = data_df.drop(data_df.loc[(data_df['type_mesure'].isin([2,4,5,6,7])) & (data_df['phase'].isin([4]))].index)
data


In [ ]:
import datetime
hr = int(datetime.datetime.now().strftime("%H"))
mn = int(datetime.datetime.now().strftime("%M"))
sd = int(datetime.datetime.now().strftime("%S"))
da_hr_ms = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
 
data['heure_mesure'] = hr
data['minute_mesure'] = mn
data['seconde_mesure'] = sd

In [ ]:
mon_dictionnaire = {'courant': int(1), 'courant_hta': int(2), 'desequilibre_courant':int(3),  
                    'tension_n':int(4), 'tension_ph':int(5), 'tension_ph_hta':int(6), 
                    'desequilibre_tension':int(7)}


In [ ]:
import pickle

with open('r16_Tfo1_courant.pkl', "rb") as a:
    crt = pickle.load(a)
with open('r16_Tfo1_courant_hta.pkl', "rb") as b:
    crt_hta = pickle.load(b)
with open('r16_Tfo1_desequilibre_courant.pkl', "rb") as c:
    des_crt = pickle.load(c)
with open('r16_Tfo1_tension_n.pkl', "rb") as d:
    ten = pickle.load(d)
with open('r16_Tfo1_tension_ph.pkl', "rb") as e:
    ten_ph = pickle.load(e)
with open('r16_Tfo1_tension_ph_hta.pkl', "rb") as f:
    ten_ph_hta = pickle.load(f)
with open('r16_Tfo1_desequilibre_tension.pkl', "rb") as g:
    des_ten = pickle.load(g)


data_omo = data.copy()

data['type_mesure'].loc[data['type_mesure']==1] = crt.predict(data_omo.loc[data['type_mesure']==1])
data['type_mesure'].loc[data['type_mesure']==2] = crt_hta.predict(data_omo.loc[data_omo['type_mesure']== 2])
data['type_mesure'].loc[data['type_mesure']==3] = des_crt.predict(data_omo.loc[data_omo['type_mesure']== 3])
data['type_mesure'].loc[data['type_mesure']==4] = ten.predict(data_omo.loc[data_omo['type_mesure']== 4])
data['type_mesure'].loc[data['type_mesure']==5] = ten_ph.predict(data_omo.loc[data_omo['type_mesure']== 5 ])
data['type_mesure'].loc[data['type_mesure']==6] = ten_ph_hta.predict(data_omo.loc[data_omo['type_mesure']== 6 ])
data['type_mesure'].loc[data['type_mesure']==7] = des_ten.predict(data_omo.loc[data_omo['type_mesure']== 7 ])



In [ ]:
data['mesure'] = data['type_mesure']
data['type_mesure'] = data_omo['type_mesure']

In [ ]:
data['type_mesure'] = data['type_mesure'].map({
                             1:'courant',
                             2:'courant_hta',
                             3:'desequilibre_courant',
                             4:'tension_n',
                             5:'tension_ph',
                             6:'tension_ph_hta',
                             7:'desequilibre_tension',                    
                             },
                             na_action=None)

data['phase']= data['phase'].replace([4],['N'])


In [ ]:
data['date_heure_mesure'] = da_hr_ms
data = data.drop(columns=['heure_mesure', 'minute_mesure', 'seconde_mesure'])


In [ ]:

data['id_poste'] = 'P0016.210.043'
data['zone_poste'] = 'transfo1'
data['classification'] = 'CLA_BT'
data['classification'].loc[data['type_mesure'].isin(['tension_ph_hta', 'courant_hta'])] = 'CLA_HTA'
data['date_heure_transfert'] = da_hr_ms
data['unite_mesure'] = data['type_mesure']
data['unite_mesure'].loc[data['unite_mesure'].isin(['energie'])] = 'MWh'
data['unite_mesure'].loc[data['unite_mesure'].isin(['frequence'])] = 'Hz'
data['unite_mesure'].loc[data['unite_mesure'].isin(['puissance_active'])] = 'W'
data['unite_mesure'].loc[data['unite_mesure'].isin(['puissance_apparente'])] = 'VA'
data['unite_mesure'].loc[data['unite_mesure'].isin(['puissance_reactive'])] = 'VAR'
data['unite_mesure'].loc[data['unite_mesure'].isin(['facteur_puissance', 'taux_de_charge', 'tdh_courant', 'tdh_tension'])] = '%'
# data['phase'] = data['phase'].astype(int)
data['phase'] = data['phase'].astype(str)
data['description_mesure'] = 'mesure - ' + data['type_mesure'] + ' phase ' +  data['phase']


In [ ]:
data

In [ ]:
data = data[[
    'id_poste', 'zone_poste', 'classification', 
    'type_mesure', 'description_mesure', 'phase', 'mesure',
    'unite_mesure', 'date_heure_mesure','date_heure_transfert'
    ]]
    
data_log = data.to_json(orient = 'records', lines=True)


In [ ]:
files_name = 'R16_Tab1_{}.log'.format(datetime.datetime.now().strftime("%Y-%m-%d"))

file = open(files_name, "a+")
file.write(data_log)
file.close()


In [ ]:
test['prediction'] = reg.predict(X_test)


In [ ]:
test.loc[(test['type_mesure']==1) & (test['depart']==2)].iloc[0:60]
